## Calculating prompt embeddings for the Stable Diffusion Competition

This notebook demonstrates how to the [sentence-transformers](https://github.com/UKPLab/sentence-transformers/) library to calculate sentence embeddings.
 [[docs](https://www.sbert.net/index.html)]


**NOTE:** Since the re-run notebooks won't have internet access, you will need to attach the dataset [sentence-transformers-222](https://www.kaggle.com/datasets/inversion/sentence-transformers-222) to your notebooks to be able to install the Sentence Transformers library and access the `all-MiniLM-L6-v2` model used for encodings during the notebook re-run on the test data.

In [1]:
import sys
import numpy as np
import pandas as pd
from pathlib import Path

sys.path.append('../input/sentence-transformers-222/sentence-transformers')
from sentence_transformers import SentenceTransformer, models

comp_path = Path('/kaggle/input/stable-diffusion-image-to-prompts/')

## Actual prompts used for the images

**NOTE:** This file will *not* be available for the notebook re-run. References to it will create notebook failures.

In [2]:
prompts = pd.read_csv(comp_path / 'prompts.csv', index_col='imgId')
prompts.head(7)

,prompt
imgId,
20057f34d,hyper realistic photo of very friendly and dys...
227ef0887,"ramen carved out of fractal rose ebony, in the..."
92e911621,ultrasaurus holding a black bean taco in the w...
a4e1c55a9,a thundering retro robot crane inks on parchme...
c98f79f71,"portrait painting of a shimmering greek hero, ..."
d8edf2e40,an astronaut standing on a engaging white rose...
f27825b2c,Kaggle employee Phil at a donut shop ordering ...


## The Sample Submission contains correct embeddings (for the example images)

The `sample_submission.csv` file on the Data page has the correct imbeddings for the prompts listed in the `prompts.csv` file. This is so you can test whether you are calculating embeddings correctly.

In [3]:
sample_submission = pd.read_csv(comp_path / 'sample_submission.csv', index_col='imgId_eId')
sample_submission.head()

,val
imgId_eId,
20057f34d_0,0.018848
20057f34d_1,0.030190
20057f34d_2,0.072792
20057f34d_3,-0.000673
20057f34d_4,0.016774


## Load the embedding model `all-MiniLM-L6-v2`

This model maps sentences to a 384 dimensional dense vector space. Load it from the [dataset](https://www.kaggle.com/datasets/inversion/sentence-transformers-222).

In [4]:
st_model = SentenceTransformer('/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2')

## Calculate prompt embeddings

In [5]:
prompt_embeddings = st_model.encode(prompts['prompt']).flatten()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## Compare calculated embeddings with ground truth (within tolerance)

In [6]:
assert np.all(np.isclose(sample_submission['val'].values, prompt_embeddings, atol=1e-07))